# BayesianSearchCV

1. Apa itu hyperparameter tuning?
2. Jelaskan metode BayesSearchCV!
3. Bagaimana cara kerja BayesSearchCV?
4. Apa kelebihan BayesSearchCV dibandingkan metode hyperparameter tuning lainnya?



1. Hyperparameter adalah parameter yang ditentukan sebelum training model. Contohnya depth pada decision tree, learning rate pada gradient boosting, C/gamma pada SVM). Sehingga hyperparameter tuning ialah proses mencari kombinasi hyperparameter terbaik supaya model menghasilkan performa maksimal (misal akurasi tinggi, error rendah).
2. BayesSearchCV adalah metode hyperparameter tuning berbasis Bayesian Optimization. BayesSearcCV tersedia di library scikit-optimize (skopt). Metode ini menggunakan model probabilistik untuk memprediksi area hyperparameter mana yang kemungkinan menghasilkan hasil bagus lalu menguji area tersebut.
3. Cara kerja BayesSearcCV ialah memulai dengan beberapa kombinasi acak lalu dicoba dan dievaluasi. Bangun model probabilistik (misalnya Gaussian Process) yang memperkirakan hubungan antara hyperparameter dan skor (accuracy/error. Gunakan model tersebut untuk memprediksi area "menjanjikan" di ruang hyperparameter. Pilih kombinasi terbaik berikutnya untuk diuji dan bukan random. Ulangi langkah sebelumnya sampai jumlah iterasi tercapai. Ambil kombinasi dengan performa terbaik.
4. Kelebihan BayesSearchCV dibandingkan metode hyperparameter tuning lainnya:
*   GridSearchCV boros waktu, coba semua kombinasi walaupun tidak perlu. Namun, BayesSearchCV lebih hemat dan fokus ke area menjanjikan
*   RandomizedSearchCV lebih cepat dari grid tapi masih random dan bisa melompati kombinasi bagus. Tetapi, BayesSearchCV menggunakan informasi dari percobaan sebelumnya (adaptif)
*   Manual Tuning lebih subjektif, rawan bias, dan memakan waktu. BayesSearchCV sistematis dan terotomatisasi

In [1]:
# import library
!pip install scikit-optimize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 3.4 MB/s eta 0:00:00


In [3]:
#improt library lagi
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [4]:
# Load Dataset
X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=0)

In [5]:
# Definisikan Model dan parameter yang akan dioptimasi (Ruang Hyperparameter)
# import library
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.svm import SVC

# Definisikan model
svc = SVC()

# Definisikan ruang hyperparameter
param_space = {
    'C': Real(1e-6, 1e+6, prior='log-uniform'),
    'gamma': Real(1e-6, 1e+1, prior='log-uniform'),
    'kernel': Categorical(['linear', 'rbf', 'poly'])
}

# BayesSearchCV setup
opt = BayesSearchCV(
    estimator=svc,
    search_spaces=param_space,
    n_iter=32,
    cv=3,
    random_state=0,
    n_jobs=-1
)


Fungsi Optimisasi Menggunakan BayesianSearchCV

In [9]:
# Inisialisasi BayesSearchCV
opt = BayesSearchCV(
    estimator=svc,
    search_spaces=param_space,
    n_iter=32,
    cv=3,
    random_state=0,
    n_jobs=-1
)

In [7]:
# Jalankan optimisasi
opt.fit(X_train, y_train)

# Tampilkan hasil terbaik
print("Best Parameters:", opt.best_params_)
print("Best CV Score:", opt.best_score_)


Best Parameters: OrderedDict({'C': 0.0012602593949011189, 'gamma': 7.117469405170288, 'kernel': 'poly'})
Best CV Score: 0.9642010431484116


In [8]:
# Evaluasi Model di data uji
y_pred = opt.predict(X_test)

# Hitung akurasi
from sklearn.metrics import accuracy_score, classification_report

print("Test Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Test Accuracy: 0.9736842105263158

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       1.00      0.94      0.97        16
           2       0.90      1.00      0.95         9

    accuracy                           0.97        38
   macro avg       0.97      0.98      0.97        38
weighted avg       0.98      0.97      0.97        38

